# Advent of Code 2023: Day 10
https://adventofcode.com/2023/day/10


## Part 1
Calculate how many steps away (from the starting point) the farthest point is in a loop.

### Get the data into a list of lists of characters


In [223]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
data_map = []
for line in data_list:
  data_map.append(list(line))

### Create function to find the starting position

In [224]:
def findStart(input_map, symbol):
  for i in range(len(data_map)):
    for j in range(len(data_map[0])):
      if input_map[i][j] == symbol:
        res = [i,j]
        return res
  return 'No starting position found'

### Create function to find which directions the loop starts from
As it's a loop, it is enough to return just one direction

In [225]:
def findLoop(input_map, start):
  dirs = [[-1,0],[1,0],[0,-1],[0,1]]
  dirs_symbols = [['|', '7', 'F'],['|', 'J', 'L'],['-', 'L', 'F'],['-', '7', 'J']]
  possible = [] #[up, down, left, right]
  for i in range(4):
    if input_map[start[0]+dirs[i][0]][start[1]+dirs[i][1]] in dirs_symbols[i]:
      possible.append(1)
    else:
      possible.append(0)

  return ['u','d','l','r'][possible.index(1)]

### Create function to traverse the loop from start to finish and count the number of steps required
To find the halfway point, set the divider to 2

In [226]:
def traverseLoop(input_map, start, symbol, dir, div=1, return_loop=False):
  dirs = {
      'u': [-1,0],
      'd': [1,0],
      'l': [0,-1],
      'r': [0,1]
  }

  dirs_symbols = {
      'u': {'|': 'u',
            '7': 'l',
            'F': 'r'},
      'd': {'|' :'d',
            'J': 'l',
            'L': 'r'},
      'l': {'-': 'l',
            'L': 'u',
            'F': 'd'},
      'r': {'-': 'r',
            'J': 'u',
            '7': 'd'}
  }
  curr = start
  curr_dir = dir
  loop = {str(start): [start, symbol, dir]}
  steps = 0
  while True:
    steps +=1
    curr = [curr[0]+dirs[curr_dir][0], curr[1]+dirs[curr_dir][1]]
    if input_map[curr[0]][curr[1]] == symbol:
      if return_loop:
        return loop
      else:
        return steps // div
    loop[str(curr)] = [curr, input_map[curr[0]][curr[1]], curr_dir]
    curr_dir = dirs_symbols[curr_dir][input_map[curr[0]][curr[1]]]

### Calculate the farthest point

In [227]:
start_pos = findStart(data_map, 'S')
start_dir = findLoop(data_map, start_pos)
traverseLoop(data_map, start_pos, 'S', start_dir, 2)

6806

## Part 2
Find the number of tiles enclosed by the loop

### Create function to find out what symbol the starting 'S' should actually be

In [243]:
def findStartSymbol(loop_data):
  neighbours = [loop_data[list(loop_data.keys())[1]][0],loop_data[list(loop_data.keys())[-1]][0]]
  if neighbours[0][0] == neighbours[1][0]:
    return '-'
  elif neighbours[0][1] == neighbours[1][1]:
    return '|'
  else:
    start = loop_data[list(loop_data.keys())[0]][0]
    if [start[0]-1,start[1]] in neighbours:
      if[start[0],start[1]-1] in neighbours:
        return 'J'
      else:
        return 'L'
    elif [start[0]+1,start[1]] in neighbours:
      if [start[0],start[1]-1] in neighbours:
        return '7'
      else:
        return 'F'

### Retrieve the loop, go through all non-loop tiles and compute if they are enclosed or not enclosed by the loop
This is done by calculating how many times a line drawn from the tile to the edge of the map intersects lines of the loop. If it intersects an even number of times it is outside the loop, if an odd number it is enclosed by the loop.

In [244]:
loop = traverseLoop(data_map, start_pos, 'S', start_dir, return_loop=True)
loop_locations = list(loop.keys())
start_symbol = findStartSymbol(loop)
loop[loop_locations[0]][1] = start_symbol
tile_count = 0
for i in range(len(data_map)):
  for j in range(len(data_map[0])):
    if (i != 0) and (j != 0) and (i != len(data_map)-1) and (j != len(data_map[0])-1):
      if str([i,j]) not in loop_locations:
        l = 0
        c = 0
        while l < j:
          if str([i,l]) in loop_locations:
            if loop[str([i,l])][1] == 'L':
              while loop[str([i,l+1])][1] == '-':
                l +=1
              if loop[str([i,l+1])][1] == 'J':
                c +=1
              l +=1
            elif loop[str([i,l])][1] == 'F':
              while loop[str([i,l+1])][1] == '-':
                l +=1
              if loop[str([i,l+1])][1] == '7':
                c +=1
              l +=1
            else:
              c +=1
              l +=1
          else:
            l +=1
        if c%2 != 0:
          tile_count += 1

tile_count

449